In [3]:
from __future__ import print_function
import base64  
import pandas as pd
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from pandas.io import gbq
from oauth2client import file, client, tools

while True:


    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    


    creds = None

    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)


    label = 'INBOX'
    print('\n')
    results = service.users().messages().list(userId='me',labelIds=label).execute()
    messages = results.get('messages',[])


    From=[]
    To=[]
    DateTime= []
    Subject= []
    Body= []


    def email():

        message_count = len(messages)

        if not messages:
            print('No message found.')
        else:



            for message in messages[:message_count]:
                msg = service.users().messages().get(userId='me', id=message['id'],format = 'full', metadataHeaders=None).execute()



                payload = msg['payload'] 
                headers = payload['headers']

                for d in headers: 

                    if d['name'] == 'From': 
                        from_1 = d['value']
                    if d['name'] == 'To':
                        to = d['value']
                    if d['name'] == 'Subject': 
                        subject = d['value']




                payload = msg['payload']
                try:
                    datetime1 = payload['headers'][1]['value'].split(';')[1].strip()
                    DateTime.append(datetime1)
                except:
                        try:
                            datetime2 = payload['headers'][2]['value'].split(';')[1].strip()
                            DateTime.append(datetime2)
                        except:
                            datetime3= payload['headers'][1]['value'] 
                            DateTime.append(datetime3)


                From.append(from_1)
                To.append(to)
                Subject.append(subject)



                try: 
                    body_1 = base64.urlsafe_b64decode(msg['payload']['parts'][0]['body']['data'].encode('utf-8'))
                    body_1 = body_1.decode('utf-8')
                    Body.append(body_1)

                except:
                        try:
                            body_2 = base64.urlsafe_b64decode(msg['payload']['parts'][0]['parts'][0]['body']['data'].encode('utf-8'))
                            body_2 = body_2.decode('utf-8')
                            Body.append(body_2)
                        except:
                            body_3 = base64.urlsafe_b64decode(msg['payload']['body']['data'].encode('utf-8'))
                            body_3 = body_3.decode('utf-8')
                            Body.append(body_3)


                try:
                    attachment = base64.urlsafe_b64decode(msg['payload']['parts'][1]['body']['attachmentId'].encode('utf-8'))
                    attachment = attachment.decode('utf-8')
                except:
                    pass




                columns= ["From_address", "To", "DateTime", "Subject", "Body"]
                Data = pd.DataFrame(columns=columns)
                Data["From_address"] = From
                Data["To"] = To
                Data['DateTime']= DateTime
                Data["Subject"]= Subject
                Data['Body'] = Body
                Data.to_csv(index=False, encoding='utf-8') 
                Data.to_csv('Data.csv', encoding='utf-8')


    email()
    pd.options.display.max_colwidth = 100000000
    df = pd.read_csv('Data.csv',encoding= 'unicode_escape')
    df.drop(['Unnamed: 0', 'To'], axis='columns', inplace=True)
    df = df.replace('\]','',regex=True)
    df = df.replace('\[','',regex=True)
    df = df.replace('\<','',regex=True)
    df = df.replace('\>','',regex=True)
    df = df.replace("\'",'',regex=True)
    df = df.replace("\\r\n",'',regex=True)
    df['DateTime'] = pd.to_datetime(df['DateTime'], utc = True)
    df = df.sort_index(ascending = True)
    df.to_csv('final_Email_data.csv')


    print('Email Data has been successfully saved to the Directory')
#     df.to_gbq(destination_table = 'Gmail_emails.email_data',project_id = 'rapid-analytics-291117',if_exists = 'replace')
#     print('email data successfully updated to BigQueqy.')
    

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=412078505-gjje8m7va9ggstrlj72i5nqo4rvonhhi.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54552%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=n4zN1Hp7eH7HEgaKSwsCLdX8YFFhRR&access_type=offline


Email Data has been successfully saved to the Directory


Email Data has been successfully saved to the Directory


Email Data has been successfully saved to the Directory


Email Data has been successfully saved to the Directory


Email Data has been successfully saved to the Directory




ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [12]:
df.head()

,From_address,DateTime,Subject,Body
0,Sailesh Shocker sailesh.shocker@gmail.com,2020-11-16 17:10:25+00:00,test,testing2
1,Sailesh Shocker sailesh.shocker@gmail.com,2020-11-16 17:09:20+00:00,test,testing1
2,Google no-reply@accounts.google.com,2020-11-13 19:00:24+00:00,Security alert,"image: GoogleQuickstart was granted access to your Google Accountrapidanalytics.testing@gmail.comIf you did not grant access, you should check this activity and secure youraccount.Check activityhttps://accounts.google.com/AccountChooser?Email=rapidanalytics.testing@gmail.com&continue=https://myaccount.google.com/alert/nt/1605294023000?rfn%3D127%26rfnc%3D1%26eid%3D-72503931186097034%26et%3D0You can also go directly to:https://myaccount.google.com/notificationsYou received this email to let you know about important changes to yourGoogle Account and services.Â© 2020 Google LLC, 1600 Amphitheatre Parkway, Mountain View, CA 94043, USA"
3,Google no-reply@accounts.google.com,2020-11-13 18:58:59+00:00,Security alert,"image: GoogleQuickstart was granted access to your Google Accountrapidanalytics.testing@gmail.comIf you did not grant access, you should check this activity and secure youraccount.Check activityhttps://accounts.google.com/AccountChooser?Email=rapidanalytics.testing@gmail.com&continue=https://myaccount.google.com/alert/nt/1605293937000?rfn%3D127%26rfnc%3D1%26eid%3D-2726565386723655849%26et%3D0You can also go directly to:https://myaccount.google.com/notificationsYou received this email to let you know about important changes to yourGoogle Account and services.Â© 2020 Google LLC, 1600 Amphitheatre Parkway, Mountain View, CA 94043, USA"
4,Google no-reply@accounts.google.com,2020-11-13 18:16:36+00:00,Security alert,"image: GoogleGCP+ Gmail was granted access to your Google Accountrapidanalytics.testing@gmail.comIf you did not grant access, you should check this activity and secure youraccount.Check activityhttps://accounts.google.com/AccountChooser?Email=rapidanalytics.testing@gmail.com&continue=https://myaccount.google.com/alert/nt/1605291395000?rfn%3D127%26rfnc%3D1%26eid%3D8475594423933423734%26et%3D0You can also go directly to:https://myaccount.google.com/notificationsYou received this email to let you know about important changes to yourGoogle Account and services.Â© 2020 Google LLC, 1600 Amphitheatre Parkway, Mountain View, CA 94043, USA"
